This notebook contains the commands to export selected visualizations as HTML. Just run the notebook to save the visualizations as interactive HTML maps. Use the [visualize](./visualize.ipynb) notebook if you want to learn how to create dataframes and visualizations from PostGIS data.

In [1]:
import sys
from ipygis import get_connection_url, QueryResult, generate_map
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

from kepler_h3_config import config  # we may use our own custom visualization config

In [2]:
sql_url = get_connection_url(dbname='geoviz')
engine = create_engine(sql_url)
session = sessionmaker(bind=engine)()

sys.path.insert(0, '..')
from models import OSMPoint, OSMPolygon, FlickrPoint, GTFSStop, OSMAccessNode, OoklaPoint, KonturPoint

The next phase will take a while. Here we actually create the dataframes in memory, from the whole database.

In [3]:
eating_query = session.query(OSMPoint).filter(OSMPoint.tags['amenity'].astext.in_(
    ['restaurant','bar','pub','biergarten','cafe','fast_food','food_court','ice_cream']
))
culture_query = session.query(OSMPoint).filter(OSMPoint.tags['amenity'].astext.in_(
    ['arts_centre','cinema','community_centre','conference_centre','events_venue','nightclub','theatre']
)|(OSMPoint.tags['tourism'].astext.in_(['museum','gallery'])))
shops_query = session.query(OSMPoint).filter(OSMPoint.tags.has_key('shop'))
photos_query = session.query(FlickrPoint)
stops_query = session.query(GTFSStop)
access_points_query = session.query(OSMAccessNode)

eating = QueryResult.create(eating_query, resolution=8, name='eating')
culture = QueryResult.create(culture_query, resolution=8, name='culture')
shops = QueryResult.create(shops_query, resolution=8, name='shops')
photographers = QueryResult.create(photos_query, resolution=8, name='photographers', group_by='properties.owner')
trips = QueryResult.create(stops_query, resolution=8, name='trips', plot='sum', column='properties.ntrips')
access = QueryResult.create(access_points_query, resolution=8, name='access', plot='mean', column='accessibilities.5')

Once the QueryResults exist, calculating the sum map from dataframes should be relatively fast.

In [4]:
results = [eating, culture, shops, photographers, trips, access]
columns = ['size', 'size', 'size', 'size', 'ntrips', '5']
weights = [1, 1, 1, 1, 1, -1]
#results = [eating, culture]
#columns = ['size', 'size']
#weights = [1, 1]

In [9]:
ookla_query = session.query(OoklaPoint)
speeds = QueryResult.create(ookla_query, resolution=8, name='speeds', plot='mean', column='properties.avg_d_kbps')
devices = QueryResult.create(ookla_query, resolution=8, name='devices', plot='sum', column='properties.devices')

In [3]:
kontur_query = session.query(KonturPoint)
population = QueryResult.create(kontur_query, resolution=8, name='population', plot='sum', column='properties.population')

In [4]:
results = [population]
columns = ['population']
weights = [1]

In [5]:
result_map = generate_map(results, 500, config=config, column=columns, weights=weights)
result_map.save_to_html()
result_map

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter
Map saved to keplergl_map.html!


KeplerGl(config={'version': 'v1', 'config': {'visState': {'filters': [], 'layers': [{'id': 'hex8_population', …

In [7]:
population.gdf

,size,geometry,hex8
hex8,,,
881126daddfffff,1,POINT (25.0643072292023348 60.1523554922848618),881126daddfffff
881126dadbfffff,1,POINT (25.0703690561534067 60.1656013096080358),881126dadbfffff
881126dad9fffff,1,POINT (25.0607402808084814 60.1598585079772477),881126dad9fffff
881126dad7fffff,1,POINT (25.0871297965543150 60.1563356810106313),881126dad7fffff
881126dad5fffff,1,POINT (25.0774991801554847 60.1505939377351027),881126dad5fffff
...,...,...,...
8808996889fffff,1,POINT (24.7788194363499876 60.2321608755370690),8808996889fffff
8808996887fffff,1,POINT (24.8052182367291678 60.2287093198884591),8808996887fffff
8808996885fffff,1,POINT (24.7956422524874505 60.2229528410028010),8808996885fffff


In [7]:
eating.gdf

,size,geometry,hex8
hex8,,,
88098a7933fffff,1,POINT (9.93362 60.24170),88098a7933fffff
880999b115fffff,1,POINT (9.93934 60.03689),880999b115fffff
880999a0e1fffff,1,POINT (10.22517 60.11204),880999a0e1fffff
880999a819fffff,1,POINT (10.32643 60.01748),880999a819fffff
880999a9d7fffff,1,POINT (10.41718 60.00552),880999a9d7fffff
...,...,...,...
8811068c61fffff,1,POINT (29.89642 59.88930),8811068c61fffff
881106a911fffff,1,POINT (30.35183 60.00927),881106a911fffff
8811044113fffff,1,POINT (29.56856 60.25292),8811044113fffff
